In [129]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import matplotlib.pyplot as plt
import re
import cv2
import os
import urllib
import numpy as np
import pandas as pd
import sqlite3

headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
    else: #bot 접근 못하게 막은데있어어 else뺌
        try:
            resp = requests.request(method, url,
                   params=params if method=='GET' else {},
                   data=params if method=='POST' else {},
                   headers=headers)
            resp.raise_for_status()
        except HTTPError as e:
            if limit > 0 and e.response.status_code >= 500: # 500 에러 재귀 ;너무많은 트래픽 안생기도록
                print(limit)
                time.sleep(300) # => random, 1초에 1번
                resp = download(url, params, headers, method, limit-1)
            else:
                print('[{}] '.format(e.response.status_code) + url)
                print(e.response.status_code)
                print(e.response.reason)
                print(e.response.headers)
        return resp
    
def letterbox(img, new_shape=(240, 320), color=(114, 114, 114)):
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])

    ratio = r, r  
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  
    # minimum rectangle
    dw, dh = np.mod(dw, 64), np.mod(dh, 64)  # wh padding

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

def labelvs(k):
    if k <200:
        a = 'Dense'
    elif k < 1000:
        a = 'Shallow'
    else:
        a = 'None'
    return a

def weatherlabel(data):
    if data['강수15'] != 0:
        return '비'
    elif data['fog'] == 'Dense' or data['fog'] == 'Shallow' :
        return '안개'
    else:
        return '맑음'
    

def sunnylabel(k):
    if k == '맑음':
        return 1
    return 0

In [2]:
import datetime

In [3]:
today = datetime.datetime.today()
today1 = today.strftime('%Y%m%d%H')

In [130]:
url = 'http://openapi.its.go.kr:8081/api/NCCTVInfo'
api_key = '1602398376506'
params= {
    'key':api_key,
    'ReqType':'2',
    'type ':'its',
    'MinX' : 126.449010,
    'MaxX' : 127.340151,
    'MinY' : 37.008739,
    'MaxY' : 37.773444,
}
resp=download(url,params, headers,'POST')

URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>

In [ ]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
col = ['roadsectionid','filecreatetime','cctvtype','cctvurl','cctvresolution','coordy','cctvformat','cctvname','coordx']
row = []
rows = []
for i in dom.find_all('data'):
    for co in col:
        k = i.find(co).text.strip()
        row.append(k)
    rows.append(row)
    row = []

In [ ]:
import pandas as pd
data = pd.DataFrame(rows,columns=col)
data.head()

In [63]:
data['coordy'] = data['coordy'].astype('float32')
data['coordx'] = data['coordx'].astype('float32')
sum(data['cctvname'].str.contains('\/'))
data['cctvname'] = data['cctvname'].str.replace('/','_')
sum(data['cctvname'].str.contains('\/'))

0

In [64]:
for i in col:
    print(sum(data[i]==''))

538
538
0
0
538
0
0
0
0


C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


In [65]:
data = data.drop(['roadsectionid','filecreatetime','cctvresolution','cctvformat','cctvtype'],axis=1)
data.head()

,cctvurl,coordy,cctvname,coordx
0,http://cctvsec.ktict.co.kr/1/biQVXUi6lrO0Cl/2B...,37.406651,[수도권제1순환선] 판교분기점2,127.097061
1,http://cctvsec.ktict.co.kr/2/Do71Nvv+HamcAfOiT...,37.428890,[수도권제1순환선] 성남,127.123611
2,http://cctvsec.ktict.co.kr/3/9sQQra/DP1w1VIjZu...,37.439091,[수도권제1순환선] 성남영업소,127.122620
3,http://cctvsec.ktict.co.kr/4/Ishh8oNCksfXOlFzQ...,37.474998,[수도권제1순환선] 송파,127.129440
4,http://cctvsec.ktict.co.kr/5/azcq8e7d5lOPfYjjx...,37.511669,[수도권제1순환선] 서하남,127.149719


In [76]:
data[data['cctvname']=='[서울외곽선] 서하남']

,cctvurl,coordy,cctvname,coordx


In [77]:
data['cctvname'].value_counts()[:10]

[안양성남선] 청계산3터널(안양)        2
[안양성남선] 청계산4터널(성남)        2
[안양성남선] 청계산3터널(성남)        2
[수도권제1순환선] 서하남            2
[제2외곽순환선] 북청라대교           2
[안양성남선] 삼성산터널(안양)         2
[경부선] 안성분기점               2
[안양성남선] 삼성산터널(성남)         2
[영동선] 마성1                 1
[영동선] [인천1]마성터널(인천1 4)    1
Name: cctvname, dtype: int64

In [32]:
data.to_csv('CCTV.csv',index=False,encoding='CP949')

In [66]:
if not os.path.isdir(today1):
    os.makedirs(today1)
for url,name in zip(data['cctvurl'],data['cctvname']):
    urllib.request.urlretrieve(url,today1+'/'+ name + '.mp4')

In [28]:
data.head()

,cctvurl,coordy,cctvname,coordx,cctvdir
0,http://cctvsec.ktict.co.kr/1/biQVXUi6lrO0Cl/2B...,37.40665,[수도권제1순환선] 판교분기점2,127.09706,2020102917/[수도권제1순환선] 판교분기점2.mp4
1,http://cctvsec.ktict.co.kr/2/Do71Nvv+HamcAfOiT...,37.42889,[수도권제1순환선] 성남,127.12361,2020102917/[수도권제1순환선] 성남.mp4
2,http://cctvsec.ktict.co.kr/3/9sQQra/DP1w1VIjZu...,37.43909,[수도권제1순환선] 성남영업소,127.12262,2020102917/[수도권제1순환선] 성남영업소.mp4
3,http://cctvsec.ktict.co.kr/4/Ishh8oNCksfXOlFzQ...,37.475,[수도권제1순환선] 송파,127.12944,2020102917/[수도권제1순환선] 송파.mp4
4,http://cctvsec.ktict.co.kr/5/azcq8e7d5lOPfYjjx...,37.51167,[수도권제1순환선] 서하남,127.14972,2020102917/[수도권제1순환선] 서하남.mp4


In [67]:
n = len(os.listdir(today1))
imgs = [None]*n
for i , _ in enumerate(os.listdir(today1)):
    img = [None]*10
    cap = cv2.VideoCapture(today1 + '/' + _)
    assert cap.isOpened(), 'Failed to open %s' % _
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    for j in range(10):
        ret, image = cap.read()
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img[j] = gray
    s = np.stack([letterbox(x)[0] for x in img], 0) 
    imgs[i] = s

In [69]:
for x in imgs:
    if x.shape != (10,240,320,3):
        print(x.shape)

In [106]:
today1

'2020110207'

In [115]:
url = 'https://www.weather.go.kr/cgi-bin/aws/nph-aws_txt_min_guide_test?'+ today1 +'00&MINDB_01M&0&m&K'
resp=download(url,  headers = headers,method= 'POST')
dom = BeautifulSoup(resp.content, 'lxml')
k =pd.read_html(str(dom.find('table')),header=0)
today_weather = k[1][['AWS 지점 / 고도','AWS 지점 / 고도.1','강수','강수15','위치']]

In [116]:
today_weather

,AWS 지점 / 고도,AWS 지점 / 고도.1,강수,강수15,위치
0,90,속초,○,0,강원도 고성군 토성면 봉포리
1,92,양양(공),-,0,강원도 양양군 손양면 동호리
2,93,북춘천,○,0,강원도 춘천시 신북읍 산천리
3,95,철원,○,.,강원도 철원군 갈말읍 군탄리
4,96,독도,○,0,경상북도 울릉군 울릉읍 독도리
...,...,...,...,...,...
706,973,화순능주,○,0,전라남도 화순군 능주면 만수리
707,974,대곡,○,0,경상남도 진주시 대곡면 단목리
708,977,오창가곡,○,0,충청북도 청주시청원구 오창읍
709,978,춘천신북,○,0,강원도 춘천시 신북읍 유포리


In [123]:
today_weather.to_csv('AWS_measure.csv',index=False,encoding='CP949')

In [117]:
con = sqlite3.connect("meta_aws.db")
cur = con.cursor()
query = cur.execute('select * from test')
cols = [column[0] for column in query.description]
ee = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)
cur.close()
con.close()
ee

,index,지점,지점명,지점주소,관리관서,위도,경도
0,0,116,관악(레),경기도 과천시중앙로 관악산길(관악산기상레이더관측소),None,37.4453,126.9640
1,1,326,용문산,경기도 양평군용문면 중원리464,None,37.5455,127.6113
2,2,400,강남,서울특별시 강남구일원동 580탄천 물재생센터,None,37.4982,127.0816
3,3,401,서초,서울특별시 서초구서초동 1416번지서초 IC,None,37.4846,127.0260
4,4,402,강동,서울특별시 강동구고덕로 183서울종합직업전문학교,None,37.5555,127.1450
...,...,...,...,...,...,...,...
90,90,665,무의도,인천광역시 중구무의동 161번지무의초등학교,인천기상대(112),37.3904,126.4252
91,91,692,백학,경기도 연천군백학면 두일리66 육군 제25사단,None,38.0404,126.9245
92,92,889,현충원,서울특별시 동작구현충로 210국립서울현충원,None,37.5004,126.9765
93,93,966,풍도,None,인천기상대(112),37.1161,126.3856


In [119]:
AWS = pd.merge(today_weather,ee,left_on='AWS 지점 / 고도.1',right_on='지점명')[['AWS 지점 / 고도','AWS 지점 / 고도.1','강수','강수15','위치','위도','경도']]
AWS

,AWS 지점 / 고도,AWS 지점 / 고도.1,강수,강수15,위치,위도,경도
0,116,관악(레),○,0,경기도 과천시 중앙동,37.4453,126.9640
1,326,용문산,○,0,경기도 양평군 용문면 중원리,37.5455,127.6113
2,400,강남,○,0,서울특별시 강남구 일원동,37.4982,127.0816
3,401,서초,○,0,서울특별시 서초구 서초동,37.4846,127.0260
4,402,강동,○,0,서울특별시 강동구 고덕동,37.5555,127.1450
...,...,...,...,...,...,...,...
86,665,무의도,○,0,인천광역시 중구 무의동,37.3904,126.4252
87,692,백학,○,0,경기도 연천군 백학면 두일리,38.0404,126.9245
88,889,현충원,○,0,서울특별시 동작구 사당동,37.5004,126.9765
89,966,풍도,-,0,경기도 안산시단원구 대부남동,37.1161,126.3856


In [ ]:
servicekey = 'yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D'
# url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D&ServiceKey=yZzD%2BV9ZD4mghiOjsejpcg37tA2iS86bBY7vs6iCZLXqzC4S%2Fl9U48LbKMDx53q0tqzYXYXoyGuOA7t3tllg9g%3D%3D&numOfRows=100&dataType=JSON&dataCd=ASOS&dateCd=HR&startDt=20201025&startHh=01&endDt=20201025&endHh=23&stnIds='

con = sqlite3.connect("meta_aws.db")

cur = con.cursor()
query = cur.execute('select * from meta_asos')
cols = [column[0] for column in query.description]
rr = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)
cur.close()
con.close()

asos = pd.DataFrame()
for _ in rr['지점']:
    resp=download(url + str(_),headers = headers,method = 'GET')
    result= resp.json()
    asos_ = pd.DataFrame.from_dict(result['response']['body']['items']['item'])
    asos_ = asos_[['stnNm','tm','vs','stnId']]
    asos = pd.concat([asos,asos_])
asos.reset_index(drop=True, inplace=True)
asos['vs'] = asos['vs'].astype('int')

In [ ]:
asos['fog'] = asos['vs'].map(lambda x : labelvs(x))
ASOS = pd.merge(asos, rr, left_on='stnNm', right_on='지점명')[['지점','지점명','tm','fog','위도','경도']]
ASOS['tm'] = pd.to_datetime(ASOS['tm'])
ASOS['tm'] = ASOS['tm'].dt.strftime('%Y%m%d%H')
ASOS = ASOS[ASOS['tm']=='2020102507']
ASOS

In [120]:
from math import sin, cos, sqrt, atan2, radians
import numpy as np

def mydistance(a1,b1,a2,b2):
    x1 = radians(a1)
    y1 = radians(b1)
    x2 = radians(a2)
    y2 = radians(b2)
    dlon = x2 - x1
    dlat = y2 - y1
    a = sin(dlat / 2)**2 + cos(y1) * cos(y2) * sin(dlon / 2)**2 
    c = 2 * atan2(sqrt(a), sqrt(1 - a))    
    return c

data['AWS 지점'] = ''
data['ASOS 지점'] = ''
num = 0
for i,j in zip(data['coordx'],data['coordy']):
    distance = pd.Series(AWS[['위도','경도']].apply (lambda row: mydistance(i,j,row['경도'],row['위도']), axis=1))
    data.loc[num,'AWS 지점'] = AWS.loc[np.argmin(distance),'AWS 지점 / 고도.1']
    distance = pd.Series(ASOS[['위도','경도']].apply (lambda row: mydistance(i,j,row['경도'],row['위도']), axis=1))
    data.loc[num,'ASOS 지점'] = ASOS.loc[np.argmin(distance),'지점명']
    num+=1
    
data = pd.merge(data,AWS,left_on = 'AWS 지점',right_on = 'AWS 지점 / 고도.1')
data = pd.merge(data,ASOS,left_on = 'ASOS 지점',right_on = '지점명')
data.head()

NameError: name 'data' is not defined

In [80]:
asos['fog'] = asos['vs'].map(lambda x : labelvs(x))

In [89]:
ASOS = pd.merge(asos, rr, left_on='stnNm', right_on='지점명')[['지점','지점명','tm','fog','위도','경도']]

In [96]:
ASOS['tm'] = pd.to_datetime(ASOS['tm'])
ASOS['tm'] = ASOS['tm'].dt.strftime('%Y%m%d%H')
ASOS = ASOS[ASOS['tm']=='2020102507']

In [97]:
ASOS

,지점,지점명,tm,fog,위도,경도
6,98,동두천,2020102507,None,37.9019,127.0607
29,99,파주,2020102507,None,37.8859,126.7665
52,102,백령도,2020102507,None,37.9740,124.7124
75,108,서울,2020102507,None,37.5714,126.9658
98,112,인천,2020102507,None,37.4777,126.6249
121,119,수원,2020102507,shallow,37.2575,126.9830
144,201,강화,2020102507,None,37.7074,126.4463
167,202,양평,2020102507,None,37.4886,127.4945
190,203,이천,2020102507,shallow,37.2640,127.4842


In [98]:
AWS

NameError: name 'AWS' is not defined